# Build Treasury Quotes CRSP

### CRSP Treasury Manual

https://www.crsp.org/wp-content/uploads/guides/CRSP_US_Treasury_Database_Guide_for_SAS_ASCII_EXCEL_R.pdf

https://www.crsp.org/products/documentation/treasno-properties

# WRDS API

In [1]:
import wrds
file_key = open("../../keys/wrds_username.txt","r")
USERNAME = file_key.read()
file_key.close()

db = wrds.Connection(wrds_username=USERNAME)
#db.create_pgpass_file()

Loading library list...
Done


In [2]:
import pandas as pd
import numpy as np
from datetime import date

***

# Database quote

In [3]:
libs = db.list_libraries()
libs_crsp = [item for item in libs if item.startswith('crsp')]
treasury_databases = db.list_tables('crsp_m_treasuries')

In [4]:
DATE = '2024-05-31'
SQL_QUERY = f'select * from crsp_m_treasuries.tfz_dly where caldt=\'{DATE}\''
data_quotes = db.raw_sql(SQL_QUERY)
data_quotes

,kytreasno,kycrspid,caldt,tdbid,tdask,tdnomprc,tdnomprc_flg,tdsourcr,tdaccint,tdretnua,tdyld,tdduratn,tdpubout,tdtotout,tdpdint,tdidxratio,tdidxratio_flg
0,204084.0,20250215.107620,2024-05-31,101.890625,101.921875,101.906250,M,I,2.220467,-0.000099,0.000130,253.329453,3333.0,9509.0,0.0,NaN,None
1,204091.0,20271115.106120,2024-05-31,104.867188,104.914062,104.890625,M,I,0.266304,0.001759,0.000124,1157.539245,7775.0,22021.0,0.0,NaN,None
2,204095.0,20290815.106120,2024-05-31,107.312500,107.359375,107.335938,M,I,1.783654,0.002595,0.000123,1640.635382,4014.0,11179.0,0.0,NaN,None
3,204098.0,20360215.104500,2024-05-31,100.742188,100.835938,100.789062,M,I,1.310440,0.003192,0.000119,3332.251249,8366.0,26397.0,0.0,NaN,None
4,204100.0,20370515.105000,2024-05-31,105.210938,105.273438,105.242188,M,I,0.217391,0.004445,0.000121,3566.621871,6423.0,21413.0,0.0,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441,208227.0,20260531.204870,2024-05-31,99.984375,100.019531,100.001953,M,I,0.000000,0.000997,0.000132,704.379966,NaN,73097.0,0.0,NaN,None
442,208228.0,20270515.204500,2024-05-31,99.484375,99.515625,99.500000,M,I,0.195652,0.001693,0.000127,1020.424448,NaN,65289.0,0.0,NaN,None
443,208229.0,20290531.204500,2024-05-31,99.941406,99.988281,99.964844,M,I,0.000000,0.002664,0.000122,1655.360799,NaN,74156.0,0.0,NaN,None
444,208221.0,20241001.400000,2024-05-31,98.209667,98.213083,98.211375,M,I,0.000000,0.000166,0.000147,123.000000,NaN,NaN,0.0,NaN,None


In [5]:
KEYS = tuple(data_quotes['kytreasno'])
SQL_QUERY = f'select * from crsp_m_treasuries.tfz_iss where kytreasno in {KEYS}'
data_iss = db.raw_sql(SQL_QUERY)
data_iss

,kytreasno,kycrspid,crspid,tcusip,tdatdt,tmatdt,iwhy,tcouprt,tnippy,tvalfc,...,tfcaldt,tnotice,iymcn,itype,iuniq,itax,iflwr,tbankdt,tstripelig,tfrgntgt
0,204083.0,20241115.107500,20241115.107500,912810ES,1994-05-15,2024-11-15,0.0,7.500,2.0,3.7500,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
1,204084.0,20250215.107620,20250215.107620,912810ET,1995-02-15,2025-02-15,0.0,7.625,2.0,3.8125,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
2,204085.0,20250815.106870,20250815.106870,912810EV,1995-08-15,2025-08-15,0.0,6.875,2.0,3.4375,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
3,204086.0,20260215.106000,20260215.106000,912810EW,1996-02-15,2026-02-15,0.0,6.000,2.0,3.0000,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
4,204087.0,20260815.106750,20260815.106750,912810EX,1996-08-15,2026-08-15,0.0,6.750,2.0,3.3750,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441,208229.0,20290531.204500,20290531.204500,91282CKT,2024-05-31,2029-05-31,0.0,4.500,2.0,2.2500,...,None,0.0,None,2.0,0.0,1.0,1.0,None,None,None
442,208230.0,20310531.204620,20310531.204620,91282CKU,2024-05-31,2031-05-31,0.0,4.625,2.0,2.3125,...,None,0.0,None,2.0,0.0,1.0,1.0,None,None,None
443,208231.0,20340515.204370,20340515.204370,91282CKQ,2024-05-15,2034-05-15,0.0,4.375,2.0,2.1875,...,None,0.0,None,2.0,0.0,1.0,1.0,None,None,None
444,208232.0,20440515.104620,20440515.104620,912810UB,2024-05-15,2044-05-15,0.0,4.625,2.0,2.3125,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None


In [6]:
rawdata = data_quotes.merge(data_iss, on='kytreasno')
rawdata

,kytreasno,kycrspid_x,caldt,tdbid,tdask,tdnomprc,tdnomprc_flg,tdsourcr,tdaccint,tdretnua,...,tfcaldt,tnotice,iymcn,itype,iuniq,itax,iflwr,tbankdt,tstripelig,tfrgntgt
0,204084.0,20250215.107620,2024-05-31,101.890625,101.921875,101.906250,M,I,2.220467,-0.000099,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
1,204091.0,20271115.106120,2024-05-31,104.867188,104.914062,104.890625,M,I,0.266304,0.001759,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
2,204095.0,20290815.106120,2024-05-31,107.312500,107.359375,107.335938,M,I,1.783654,0.002595,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
3,204098.0,20360215.104500,2024-05-31,100.742188,100.835938,100.789062,M,I,1.310440,0.003192,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
4,204100.0,20370515.105000,2024-05-31,105.210938,105.273438,105.242188,M,I,0.217391,0.004445,...,None,0.0,None,1.0,0.0,1.0,1.0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441,208227.0,20260531.204870,2024-05-31,99.984375,100.019531,100.001953,M,I,0.000000,0.000997,...,None,0.0,None,2.0,0.0,1.0,1.0,None,None,None
442,208228.0,20270515.204500,2024-05-31,99.484375,99.515625,99.500000,M,I,0.195652,0.001693,...,None,0.0,None,2.0,0.0,1.0,1.0,None,None,None
443,208229.0,20290531.204500,2024-05-31,99.941406,99.988281,99.964844,M,I,0.000000,0.002664,...,None,0.0,None,2.0,0.0,1.0,1.0,None,None,None
444,208221.0,20241001.400000,2024-05-31,98.209667,98.213083,98.211375,M,I,0.000000,0.000166,...,None,0.0,None,4.0,0.0,1.0,1.0,None,None,None


***

# Process Data

In [7]:
import sys
sys.path.insert(0, '../cmds')
from treasury_cmds import process_wrds_treasury_data, select_maturities

data = process_wrds_treasury_data(rawdata)

In [8]:
idx = select_maturities(data)
data_select = data.loc[idx]

***

# Save to Excel

In [9]:
### raw data
outfile = f'../data/treasury_quotes_crsp_{DATE}.xlsx'
with pd.ExcelWriter(outfile) as writer:  
    rawdata.to_excel(writer, sheet_name= 'quotes', index=False)


### processed data
outfile = f'../data/treasury_quotes_clean_{DATE}.xlsx'
with pd.ExcelWriter(outfile) as writer:  
    data.to_excel(writer, sheet_name= 'quotes', index=True)
    data_select.to_excel(writer, sheet_name = 'selected quotes', index=True)